In [1]:
import pprint
import requests
import sys

In [2]:
api_key = "9b656c6f5237727cdf1e35288a7afa8c"

loc = [
  'Buenos Aires, Argentina',
  'Guangzhou, China',
  'Wichita, Kansas',
  'Niskayuna, New York',
  'Gwangmyeong, South Korea',
  'Taipei, Taiwan',
  'Nanaimo, British Columbia',
  'Chennai, India',
  'Barrington, Illinois',
  'Littleton, Colorado',
  'Peterhead, Scotland',
  'Vizag, India',
  'Des Moines, Iowa',
  'Beijing, China',
  'Killeen, Texas',
  'Morehead City, North Carolina'
]

city = loc[-1]

In [3]:
geo_URL = 'http://api.openweathermap.org/geo/1.0/direct'

### add below loop when finished 
### for city in loc:
geo = f'{geo_URL}?q={city}&limit=5&appid={api_key}'
resp = requests.get( geo )

if resp.status_code != 200: # Failure?
    print( f'Error geocoding {city}: {resp.status_code}' )
    sys.exit( 1 )

# OpenWeatherMap returns a list of matching cities, up to the limit specified
# in the API call; even if you only ask for one city (limit=5), it's still
# returned as a 1-element list

if len( resp.json() ) == 0: # No such city?
    print( f'Error locating city {city}; {resp.status_code}' )
    sys.exit( 2 )

json = resp.json()
if type( json ) == list: # List of cities?
    lat = json[ 0 ][ 'lat' ]
    lon = json[ 0 ][ 'lon' ]
else: # Unknown city?
    print( f'Error, invalid data returned for city {city}, {resp.status_code}' )
    sys.exit( 3 )

# Use Peterhead's latitude and longitude to get its 5-day forecast in 3-hour
# blocks

forecast_URL = 'http://api.openweathermap.org/data/2.5/forecast'
forecast = f'{forecast_URL}?lat={lat}&lon={lon}&appid={api_key}'
resp = requests.get( forecast )

if resp.status_code != 200: # Failure?
    print( f'Error retrieving data: {resp.status_code}' )
    sys.exit( 4 )

# Pretty-print the resulting JSON forecast for the first 3 hour block

print( f'{city}:' )
data = resp.json()
printer = pprint.PrettyPrinter( width=80, compact=True )
printer.pprint( data[ 'list' ][ 0 ] )

Morehead City, North Carolina:
{'clouds': {'all': 33},
 'dt': 1720418400,
 'dt_txt': '2024-07-08 06:00:00',
 'main': {'feels_like': 303.25,
          'grnd_level': 1016,
          'humidity': 83,
          'pressure': 1017,
          'sea_level': 1017,
          'temp': 300.18,
          'temp_kf': 0.49,
          'temp_max': 300.18,
          'temp_min': 299.69},
 'pop': 0.1,
 'sys': {'pod': 'n'},
 'visibility': 10000,
 'weather': [{'description': 'scattered clouds',
              'icon': '03n',
              'id': 802,
              'main': 'Clouds'}],
 'wind': {'deg': 230, 'gust': 5.28, 'speed': 3.05}}


In [49]:
from datetime import datetime
import pandas as pd

data_list = []
for i in range(24):
    forecast_data = data[ 'list' ][ i ]
    dt = datetime.strptime(forecast_data['dt_txt'], '%Y-%m-%d %H:%M:%S')
    temp_max = forecast_data['main']['temp_max']
    temp_min = forecast_data['main']['temp_min']
    data_list.append({
        'city': city,
        'datetime': dt,
        'temp_max': temp_max,
        'temp_min': temp_min
    })

df = pd.DataFrame(data_list)
df.head(10)

,city,datetime,temp_max,temp_min
0,"Morehead City, North Carolina",2024-07-08 06:00:00,300.18,299.69
1,"Morehead City, North Carolina",2024-07-08 09:00:00,299.44,298.95
2,"Morehead City, North Carolina",2024-07-08 12:00:00,299.55,299.55
3,"Morehead City, North Carolina",2024-07-08 15:00:00,301.31,301.31
4,"Morehead City, North Carolina",2024-07-08 18:00:00,301.40,301.40
5,"Morehead City, North Carolina",2024-07-08 21:00:00,301.15,301.15
6,"Morehead City, North Carolina",2024-07-09 00:00:00,300.31,300.31
7,"Morehead City, North Carolina",2024-07-09 03:00:00,300.09,300.09
8,"Morehead City, North Carolina",2024-07-09 06:00:00,299.66,299.66
9,"Morehead City, North Carolina",2024-07-09 09:00:00,298.90,298.90


In [52]:
df_grouped = df.groupby(pd.Grouper(key='datetime', axis=0, freq='D')).aggregate({'city': lambda x: x.iloc[0],'temp_max': 'max', 'temp_min': 'min'})
df_grouped['min_avg'] = df_grouped['temp_min'].mean()
df_grouped['max_avg'] = df_grouped['temp_max'].mean()
df_grouped

,city,temp_max,temp_min,min_avg,max_avg
datetime,,,,,
2024-07-08,"Morehead City, North Carolina",301.40,298.95,299.5525,301.1025
2024-07-09,"Morehead City, North Carolina",301.11,298.90,299.5525,301.1025
2024-07-10,"Morehead City, North Carolina",301.31,299.77,299.5525,301.1025
2024-07-11,"Morehead City, North Carolina",300.59,300.59,299.5525,301.1025
